# `OpenMC` example

### Importing the Python modules necessary to run OpenMC

In [3]:
import os
os.environ['OPENMC_CROSS_SECTIONS'] = '/home/nacho/openmc/data/endfb-vii.1-hdf5/cross_sections.xml'
import numpy as np
import matplotlib.pyplot as plt
import openmc
import pandas as pd
from matplotlib.colors import LogNorm
from PIL import Image   
from uncertainties import ufloat, unumpy
import h5py


### Materials

Materials are defined by calling [`openmc.Material()`](https://docs.openmc.org/en/stable/pythonapi/generated/openmc.Material.html#openmc-material) to create the material object and then using the `mat.add_nuclide()` or `mat.add_element()` methods to build the composition.

The first parameter is the nuclide or element, the second one is the fraction, and the third one is the units.

Compositions can be expressed in percentage of atoms (`ao`), percentage of mass (`wo`) or atom density (`at/cm3`).

To se the total density of the material, the module `.set_density()` should be called. The possible units are `g/cm3`, `atom/b-cm`, `kg/m3` or `sum` (asumes that the concentrations of the nuclides/elements are in `atom/b-cm`)

It is possible to ad an $S(\alpha,\beta)$ library, which will replace the cross sections for low energies.



In [4]:
deu = openmc.Material(1, 'deuterio', 293.5943174)
deu.add_nuclide('H2', 1, 'ao')
deu.set_density('atom/cm3', (985796446e13/0.0165)*0.01)
print(deu)



hid = openmc.Material(2, 'hidrogeno', 293.5943174)
hid.add_nuclide('H1', 1, 'ao')
hid.set_density('g/cm3', 0.01)
print(hid)

Material
	ID             =	1
	Name           =	deuterio
	Temperature    =	293.5943174
	Density        =	5.974523915151516e+21 [atom/cm3]
	Volume         =	None [cm^3]
	S(a,b) Tables  
	Nuclides       
	H2             =	1            [ao]

Material
	ID             =	2
	Name           =	hidrogeno
	Temperature    =	293.5943174
	Density        =	0.01 [g/cm3]
	Volume         =	None [cm^3]
	S(a,b) Tables  
	Nuclides       
	H1             =	1            [ao]



### Geometry

The geometry is defined using [Constructive Solid Geometry](https://en.wikipedia.org/wiki/Constructive_solid_geometry) (CSG). First, a set of simple surfaces are defined. Mathematically, the surfaces are defined as $f(x,y,z)=0$, giving the parameters of the function. Each surface defines two regions, one where $f(x,y,z)>0$, and one where $f(x,y,z)<0$. By combining the regions using the union (`|`), intersection (`&`) and complement (`~`) we can define regions of the space.

The basic unit of the geometry in OpenMC is the *Cell*, which is a region of space with something that fills it.

The surfaces with `vacuum` boundary conditions means that the particles that reach this surface will be eliminated. By default, the `transmission` boundary is setted. Other options are `reflective`, `periodic` and `white`.

A collection of cells is a *Universe*, and it can be used to fill other cells for nested geometries.

The surface units are in `cm`

#### Surfaces

In [5]:
L=20
xtop=openmc.XPlane(x0=L/2, boundary_type='vacuum', surface_id=1)
xbot=openmc.XPlane(x0=-L/2, boundary_type='vacuum', surface_id=2)
ytop=openmc.YPlane(y0=L/2, boundary_type='vacuum', surface_id=3)
ybot=openmc.YPlane(y0=-L/2, boundary_type='vacuum', surface_id=4)
ztop=openmc.ZPlane(z0=L/2, boundary_type='vacuum', surface_id=5)
zbot=openmc.ZPlane(z0=-L/2, boundary_type='vacuum', surface_id=6)



#### Regions

In [6]:
region=-xtop & +xbot & -ytop & +ybot & -ztop & +zbot


#### Cells

In [7]:
cell=openmc.Cell(region=region,fill=deu,cell_id=1)


#### Universe(s)

In [8]:
univ01 =openmc.Universe(cells= [cell], universe_id=1)

### Tallies (optional)

With the `Tallies` module, we can define what we want to compute from the simulations. These tallies count a specific *Score* discrimined by different *Filters*.

#### Filters

In [9]:
energies = np.linspace(0.0, 0.3, 1001)


tallies = openmc.Tallies()
current_nomatter = openmc.Tally(name='Fdet')
current_nomatter.scores = ['current']
current_nomatter.filters = [openmc.EnergyCharFilter(energies), openmc.EnergyFilter([0.0, 8.617333e-5*294.0]), openmc.SurfaceFilter(zbot)]
tallies.append(current_nomatter)


### Settings

We will now define the settings for the simulation:


* What type of simulation we want to run (fixed source or multiplication eigenvalue)
* How many particles we want to simulate
* The distribution of the source in space, angle and energy.

Also, we can specify if we want to create a file with all the particles that cross a certain surface

#### Specify the PATH for the nuclear data and the executable

In [10]:
openmc_exe= '/home/nacho/openmc/CARLITOS/build/bin/openmc'
openmc_data="/home/nacho/openmc/data/endfb-vii.1-hdf5/cross_sections.xml"

In [11]:
geom = openmc.Geometry(univ01)
geom.export_to_xml()

mats = openmc.Materials(geom.get_all_materials().values())
mats.cross_sections = openmc_data
mats.export_to_xml()

tallies.export_to_xml()


### SETINGS

The input for OpenMC is described by, at least, 3 XML files:

*   `materials.xml`: describes the materials of the system.
*   `geometry.xml`: describes the geometry of the system.
*   `settings.xml`: describes the running parameters of the system.
*   `plots.xml`: describes the plot running parameters.
*   `tallies.xml`: describes the tallies that will be computing during the simulation.

The OpenMC Python API allows us to create those files and also to postprocess the output.

We should call, for every macro-variable, the `.export_to_xml()` function

In [12]:
settings = openmc.Settings()
settings.run_mode='char-0'
settings.particles = int(1e6)
settings.batches = 10
settings.inactive = 0
settings.survival_biasing=True
settings.photon_transport=False



### CHAR-0

In [14]:
source = openmc.Source()
source.space = openmc.stats.Point((0, 0, 0))
source.angle = openmc.stats.Isotropic()
source.energy = openmc.stats.Uniform(0.0, 0.3)
settings.source = source
settings.export_to_xml()
os.system("rm summary.h5")
openmc.run(openmc_exec=openmc_exe) 


 Reading settings XML file...
 Reading cross sections XML file...
 Reading materials XML file...
 Reading geometry XML file...
 Reading H2 from /home/nacho/openmc/data/endfb-vii.1-hdf5/neutron/H2.h5
 Minimum neutron data temperature: 294 K
 Maximum neutron data temperature: 294 K
 Reading tallies XML file...
 Preparing distributed cell instances...
 Reading plot XML file...
 Writing summary.h5 file...
 Maximum neutron transport energy: 150000000 eV for H2

 =======================>     CHAR-0 SIMULATION     <=======================

 Simulating batch 1
 Simulating batch 2
 Simulating batch 3
 Simulating batch 4
 Simulating batch 5
 Simulating batch 6
 Simulating batch 7
 Simulating batch 8
 Simulating batch 9
 Simulating batch 10
Número de batch: 10
 Creating state point statepoint.10.h5...

 =======================>     TIMING STATISTICS     <=======================

 Total time for initialization     = 3.3152e-02 seconds
   Reading cross sections          = 3.0753e-03 seconds
 Total 

In [15]:
energies_mid = (energies[1:] + energies[:-1]) / 2

sp = openmc.StatePoint('statepoint.10.h5')
tally = sp.get_tally()

char_values = tally.mean.flatten()
char_std = tally.std_dev.flatten()

char_data = np.column_stack((energies_mid, char_values, char_std))
np.savetxt('char_data.txt', char_data, fmt='%.18e')

asd = np.loadtxt('char_data.txt')

print(asd.shape)
print(asd)

(1000, 3)
[[ 1.50000000e-04 -1.01079000e-04  3.37720747e-06]
 [ 4.50000000e-04 -1.14483118e-04  3.90179447e-06]
 [ 7.50000000e-04 -1.23384839e-04  2.47290035e-06]
 ...
 [ 2.99250000e-01 -1.39985379e-06  3.39902533e-07]
 [ 2.99550000e-01 -1.29987166e-06  2.99976326e-07]
 [ 2.99850000e-01 -1.09991473e-06  2.76877281e-07]]
